In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
data = pd.read_csv("data_with_labels.csv")

C:\Users\Homie\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Select Data for Feature Engineering

In [3]:
#write apply functions rather than one hot encoding

First, we have changed the datatype of question_text & answer_text columns from a long string to an actual list.

In [4]:
def change_string_to_array(data):
    data['question_text']=data['question_text'].apply(eval)
    data['answer_text']=data['answer_text'].apply(eval)
    return data

In [5]:
data['question_text'][0]

'[\'Today is your pain better, the same, or worse than it has been?\', \'If you ever need medical care, you can call us first. Do you have our <strong>&nbsp;phone number?&nbsp;</strong> {{location_name}}: {{location_phonenumber}}\', \'Do you have any <strong>&nbsp;concerns&nbsp;</strong> with your health?\', \'Have you had any <strong>&nbsp;falls&nbsp;</strong> recently?\', \'Have you met the care team from {{location_name}} who will be taking care of you?\', \'How have you been feeling lately?\', \'Do you have any questions about your <strong>&nbsp;medications?&nbsp;</strong> (Remember to not prescribe.)\', \'Do you <strong>&nbsp;understand&nbsp;</strong> how your team will be helping you?\', \'Did someone from {{location_name}} talk to you about <strong>&nbsp;fall prevention&nbsp;</strong>?\', \'Do you have a 2-week follow up <strong>&nbsp;appointment&nbsp;</strong> with your primary care physician or other physician? (Write name of physician and date of appointment in notes)\', \'Ha

In [6]:
change_string_to_array(data)

,stay_id,timestamp,readmission_risk,enacted_emergency_protocol,optional_relation,outcome,call_category_id,is_urgent,is_provider_aware,question_text,...,medications,is_callable,actual_cost,anticipated_cost,lace_score,stay_type,payer,discharge_disposition,referring_location,readmitted
0,1839118,2021-01-14 17:54:13.761181+00,1,f,NaN,1,1.0,f,NaN,"[Today is your pain better, the same, or worse...",...,NaN,t,NaN,NaN,NaN,home_health_short_term,PACIFIC SOURCE COMMERCIAL,NaN,NaN,0
1,1221101,2020-04-30 15:20:58.980727+00,1,f,NaN,1,3.0,f,NaN,"[If you ever need medical care, you can call u...",...,"[""RHOGAM - 200 mg""]",t,NaN,NaN,NaN,home_health_short_term,NaN,NaN,NaN,0
2,1220991,2020-04-22 00:13:12.255228+00,1,f,DAUGHTER,1,1.0,f,NaN,[Have you had any <strong>&nbsp;falls&nbsp;</s...,...,NaN,t,NaN,NaN,NaN,home_health_short_term,NGS - PDGM - EFFECTIVE 1/2020,NaN,NaN,0
3,2492049,2021-03-24 16:57:25.837252+00,2,f,DAUGHTER,1,1.0,f,NaN,"[Today is your pain better, the same, or worse...",...,NaN,t,NaN,NaN,NaN,home_health_short_term,NGS,NaN,NaN,0
4,1602247,2020-10-19 14:29:59.756811+00,1,f,NaN,1,1.0,f,NaN,"[Today is your pain better, the same, or worse...",...,NaN,t,NaN,NaN,NaN,snf_short_term,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489144,856557,2019-08-19 15:29:12.512955+00,1,f,NaN,1,1.0,f,NaN,[Have you had any of the following symptoms - ...,...,NaN,t,NaN,NaN,NaN,home_health_short_term,TRIWEST EPISODIC,NaN,NaN,1
489145,999954,2019-10-17 15:59:47.362389+00,4,f,MOTHER,1,1.0,f,NaN,[Do you have any <strong>&nbsp;concerns&nbsp;<...,...,"[""CRITIFIB - 20mg/ ml""]",t,NaN,NaN,NaN,home_health_short_term,NaN,NaN,NaN,1
489146,89730,2017-07-11 22:16:41.157076+00,1,f,NaN,1,1.0,f,NaN,"[How have you been feeling lately?, Do you hav...",...,NaN,f,NaN,NaN,NaN,home_health_short_term,NaN,NaN,NaN,0
489147,2398481,2021-02-23 00:55:44.560092+00,low_risk,f,NaN,1,2.0,f,NaN,[Are you interested in being vaccinated at thi...,...,NaN,t,NaN,NaN,NaN,home_health_short_term,NaN,NaN,NaN,0


In [7]:
data['question_text'][0]

['Today is your pain better, the same, or worse than it has been?',
 'If you ever need medical care, you can call us first. Do you have our <strong>&nbsp;phone number?&nbsp;</strong> {{location_name}}: {{location_phonenumber}}',
 'Do you have any <strong>&nbsp;concerns&nbsp;</strong> with your health?',
 'Have you had any <strong>&nbsp;falls&nbsp;</strong> recently?',
 'Have you met the care team from {{location_name}} who will be taking care of you?',
 'How have you been feeling lately?',
 'Do you have any questions about your <strong>&nbsp;medications?&nbsp;</strong> (Remember to not prescribe.)',
 'Do you <strong>&nbsp;understand&nbsp;</strong> how your team will be helping you?',
 'Did someone from {{location_name}} talk to you about <strong>&nbsp;fall prevention&nbsp;</strong>?',
 'Do you have a 2-week follow up <strong>&nbsp;appointment&nbsp;</strong> with your primary care physician or other physician? (Write name of physician and date of appointment in notes)',
 'Has a nurse re

## Eliminate low_risk in readmission_risk

we had reason to doubt that category 8-12 were from the old model since the readmission_risk only had low_risk scores.
However, low_risk seems to be prevalent in other categories as well. However the timestamps indicate that questions from CCID 8-12 were asked pretty recently within a year from now compared to other more prevalent categories that had older data.
so we decided to drop rows that had low_risk as readmission_risk.

In [8]:
data.groupby('call_category_id')['readmission_risk'].value_counts()

call_category_id  readmission_risk
1.0               1                   175943
                  2                    42658
                  low_risk             30296
                  4                    13565
                  3                     6537
2.0               low_risk             21706
                  1                    20014
                  2                     4545
                  4                     1504
                  3                      469
3.0               1                   129389
                  2                    11681
                  low_risk              4880
                  4                     4574
                  3                      490
4.0               low_risk                20
                  3                       13
                  2                        1
                  4                        1
5.0               low_risk              4889
                  1                     3094
                  4 

In [9]:
data.groupby('call_category_id')['timestamp'].max()

call_category_id
1.0     2021-08-16 15:48:24.620672+00
2.0     2021-08-16 16:16:31.012961+00
3.0     2021-08-16 16:15:21.232957+00
4.0     2021-08-06 15:11:12.438353+00
5.0     2021-08-16 15:03:28.857866+00
6.0     2021-08-16 15:48:51.592862+00
7.0     2021-08-04 15:25:24.160935+00
8.0     2021-08-10 20:53:02.944597+00
9.0     2020-10-26 18:18:30.666206+00
10.0    2020-11-17 20:23:50.525364+00
11.0    2020-10-22 19:23:31.407964+00
12.0    2020-12-14 20:58:10.763072+00
Name: timestamp, dtype: object

In [10]:
data.groupby('call_category_id')['timestamp'].min()

call_category_id
1.0            2016-06-04 05:18:29+00
2.0            2016-04-12 03:19:11+00
3.0     2016-11-04 00:13:00.390649+00
4.0     2020-02-03 18:36:49.691186+00
5.0            2016-04-09 08:24:09+00
6.0     2017-02-13 20:21:13.840606+00
7.0            2016-09-13 06:12:49+00
8.0     2020-05-15 20:48:48.282494+00
9.0     2020-09-24 17:48:29.980393+00
10.0    2020-09-29 18:47:19.530093+00
11.0    2020-10-01 18:31:55.857815+00
12.0    2020-09-24 18:13:49.540361+00
Name: timestamp, dtype: object

In [11]:
data['readmission_risk'].value_counts()

1           335015
low_risk     65517
2            59889
4            21059
3             7669
Name: readmission_risk, dtype: int64

In [12]:
def drop_lowrisk(data):
    no_lowrisk_data = data.drop(data[data['readmission_risk']=='low_risk'].index)
    return no_lowrisk_data
data=drop_lowrisk(data)

# converting readmission_risk is integer
data['readmission_risk'] = data['readmission_risk'].astype(int, copy=True, errors='raise')
data

,stay_id,timestamp,readmission_risk,enacted_emergency_protocol,optional_relation,outcome,call_category_id,is_urgent,is_provider_aware,question_text,...,medications,is_callable,actual_cost,anticipated_cost,lace_score,stay_type,payer,discharge_disposition,referring_location,readmitted
0,1839118,2021-01-14 17:54:13.761181+00,1,f,NaN,1,1.0,f,NaN,"[Today is your pain better, the same, or worse...",...,NaN,t,NaN,NaN,NaN,home_health_short_term,PACIFIC SOURCE COMMERCIAL,NaN,NaN,0
1,1221101,2020-04-30 15:20:58.980727+00,1,f,NaN,1,3.0,f,NaN,"[If you ever need medical care, you can call u...",...,"[""RHOGAM - 200 mg""]",t,NaN,NaN,NaN,home_health_short_term,NaN,NaN,NaN,0
2,1220991,2020-04-22 00:13:12.255228+00,1,f,DAUGHTER,1,1.0,f,NaN,[Have you had any <strong>&nbsp;falls&nbsp;</s...,...,NaN,t,NaN,NaN,NaN,home_health_short_term,NGS - PDGM - EFFECTIVE 1/2020,NaN,NaN,0
3,2492049,2021-03-24 16:57:25.837252+00,2,f,DAUGHTER,1,1.0,f,NaN,"[Today is your pain better, the same, or worse...",...,NaN,t,NaN,NaN,NaN,home_health_short_term,NGS,NaN,NaN,0
4,1602247,2020-10-19 14:29:59.756811+00,1,f,NaN,1,1.0,f,NaN,"[Today is your pain better, the same, or worse...",...,NaN,t,NaN,NaN,NaN,snf_short_term,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489143,550434,2019-01-21 17:13:56.948772+00,1,f,NaN,1,1.0,f,NaN,[Have you had any of the following symptoms - ...,...,"[""CEFUROXIME AXETIL - 220 mg""]",t,NaN,NaN,NaN,home_health_short_term,NaN,NaN,NaN,0
489144,856557,2019-08-19 15:29:12.512955+00,1,f,NaN,1,1.0,f,NaN,[Have you had any of the following symptoms - ...,...,NaN,t,NaN,NaN,NaN,home_health_short_term,TRIWEST EPISODIC,NaN,NaN,1
489145,999954,2019-10-17 15:59:47.362389+00,4,f,MOTHER,1,1.0,f,NaN,[Do you have any <strong>&nbsp;concerns&nbsp;<...,...,"[""CRITIFIB - 20mg/ ml""]",t,NaN,NaN,NaN,home_health_short_term,NaN,NaN,NaN,1
489146,89730,2017-07-11 22:16:41.157076+00,1,f,NaN,1,1.0,f,NaN,"[How have you been feeling lately?, Do you hav...",...,NaN,f,NaN,NaN,NaN,home_health_short_term,NaN,NaN,NaN,0


# Creating question banks / question answer tuples for feature generation
We sought to have columns representing each unique answer for each question asked to the patient.
In order to account for questions that are not asked, we also created a "not answered" column for every question.
we also mildly filtered out our question banks such that it wouldn't contain any questions with one unique answer.
In the future we may apply additional conditions based on popularity of the questions to weed out features that are not as essential.

In [13]:
# create a set of unique questions
def create_bank(data):
    questions=data['question_text']
    unique_q=set()
    for i in questions:
        for j in i:
            unique_q.add(j)
    return unique_q

# create question_answer pair for each row so that corresponding question and answers are formed into tuples
def create_qa_zip(data):
    data['question_answer_zip'] = data.apply(lambda x: list(zip(x['question_text'], x['answer_text'])), axis=1)
    return data

# create unique question_answer dictionary
def question_answer_combo(data):
    qa_set_dict = {}
    for qa_list in data.question_answer_zip:
        for qa_tuple in qa_list:
            if qa_tuple[0] not in qa_set_dict:
                qa_set_dict[qa_tuple[0]] = set()
                qa_set_dict[qa_tuple[0]].add(qa_tuple[1])
            else:
                qa_set_dict[qa_tuple[0]].add(qa_tuple[1])
    return qa_set_dict

# create unique question_answer dictionary for certain CCID
def question_answer_combo_ccid(data, ccid):
    qa_set_dict = {}
    for qa_list in data[data['call_category_id']==ccid].question_answer_zip:
        for qa_tuple in qa_list:
            if qa_tuple[0] not in qa_set_dict:
                qa_set_dict[qa_tuple[0]] = set()
                qa_set_dict[qa_tuple[0]].add(qa_tuple[1])
            else:
                qa_set_dict[qa_tuple[0]].add(qa_tuple[1])
    return qa_set_dict

# create unique question keys for certain CCID
def ccid_n_questions(data, ccid):
    return question_answer_combo_ccid(data, ccid).keys()

# filter out question_answer pair that only has 1 unique answer and show remaining combos
def qa_combo_condition_1(combo):
    qa_condition1={k: v for k, v in combo.items() if len(v) > 1}
    return qa_condition1

#create question bank based on condtion 1
def create_bank_condition_1(data):
    initial=question_answer_combo(data)
    useful_combo=qa_combo_condition_1(initial)
    return list(useful_combo.keys())

#create question bank that only has 1 unique answer
def useless_questions_condition_1(data):
    initial =question_answer_combo(data)
    not_qa_condition1={k: v for k, v in initial.items() if len(v) == 1}
    return list(not_qa_condition1.keys())

In [14]:
question_bank=create_bank(data)
question_bank

{'"We have appreciated the opportunity to take care of you. If you recall, we mentioned that you would be getting a satisfaction survey about {{location_name}}’s services from an outside company. When it arrives, we would appreciate you completing it."',
 '(IF patient had a positive therapy experience) Would you mind if someone from our therapy department reaches out to assess if you’re an appropriate outpatient candidate?',
 '(If wanting more home health personally) When was the last time you saw a physician? What was the purpose for that visit? Have you had any other physician visits in the last three months? What was the purpose of that/those visit(s)\n<br />\n<i>Let them know that they need to have seen a physician for the reason that they need home health. If they do not have a qualifying face to face visit let them know that they need to schedule a visit.</i>',
 "(ONLY if patient has positive feedback) Thank you so much for your positive feedback. You'll receive a short survey in

In [15]:
create_qa_zip(data)
#creates a column with a list of tuples for (question, answer)

,stay_id,timestamp,readmission_risk,enacted_emergency_protocol,optional_relation,outcome,call_category_id,is_urgent,is_provider_aware,question_text,...,is_callable,actual_cost,anticipated_cost,lace_score,stay_type,payer,discharge_disposition,referring_location,readmitted,question_answer_zip
0,1839118,2021-01-14 17:54:13.761181+00,1,f,NaN,1,1.0,f,NaN,"[Today is your pain better, the same, or worse...",...,t,NaN,NaN,NaN,home_health_short_term,PACIFIC SOURCE COMMERCIAL,NaN,NaN,0,"[(Today is your pain better, the same, or wors..."
1,1221101,2020-04-30 15:20:58.980727+00,1,f,NaN,1,3.0,f,NaN,"[If you ever need medical care, you can call u...",...,t,NaN,NaN,NaN,home_health_short_term,NaN,NaN,NaN,0,"[(If you ever need medical care, you can call ..."
2,1220991,2020-04-22 00:13:12.255228+00,1,f,DAUGHTER,1,1.0,f,NaN,[Have you had any <strong>&nbsp;falls&nbsp;</s...,...,t,NaN,NaN,NaN,home_health_short_term,NGS - PDGM - EFFECTIVE 1/2020,NaN,NaN,0,[(Have you had any <strong>&nbsp;falls&nbsp;</...
3,2492049,2021-03-24 16:57:25.837252+00,2,f,DAUGHTER,1,1.0,f,NaN,"[Today is your pain better, the same, or worse...",...,t,NaN,NaN,NaN,home_health_short_term,NGS,NaN,NaN,0,"[(Today is your pain better, the same, or wors..."
4,1602247,2020-10-19 14:29:59.756811+00,1,f,NaN,1,1.0,f,NaN,"[Today is your pain better, the same, or worse...",...,t,NaN,NaN,NaN,snf_short_term,NaN,NaN,NaN,0,"[(Today is your pain better, the same, or wors..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489143,550434,2019-01-21 17:13:56.948772+00,1,f,NaN,1,1.0,f,NaN,[Have you had any of the following symptoms - ...,...,t,NaN,NaN,NaN,home_health_short_term,NaN,NaN,NaN,0,[(Have you had any of the following symptoms -...
489144,856557,2019-08-19 15:29:12.512955+00,1,f,NaN,1,1.0,f,NaN,[Have you had any of the following symptoms - ...,...,t,NaN,NaN,NaN,home_health_short_term,TRIWEST EPISODIC,NaN,NaN,1,[(Have you had any of the following symptoms -...
489145,999954,2019-10-17 15:59:47.362389+00,4,f,MOTHER,1,1.0,f,NaN,[Do you have any <strong>&nbsp;concerns&nbsp;<...,...,t,NaN,NaN,NaN,home_health_short_term,NaN,NaN,NaN,1,[(Do you have any <strong>&nbsp;concerns&nbsp;...
489146,89730,2017-07-11 22:16:41.157076+00,1,f,NaN,1,1.0,f,NaN,"[How have you been feeling lately?, Do you hav...",...,f,NaN,NaN,NaN,home_health_short_term,NaN,NaN,NaN,0,"[(How have you been feeling lately?, Feeling g..."


In [16]:
combos=question_answer_combo(data)
combos

{'Today is your pain better, the same, or worse than it has been?': {'Pain is about the same',
  'Pain is better',
  'Pain is worse'},
 'If you ever need medical care, you can call us first. Do you have our <strong>&nbsp;phone number?&nbsp;</strong> {{location_name}}: {{location_phonenumber}}': {'Already had phone number',
  "Didn't give phone number",
  'Gave phone number'},
 'Do you have any <strong>&nbsp;concerns&nbsp;</strong> with your health?': {'Has health concerns',
  'Has no health concerns'},
 'Have you had any <strong>&nbsp;falls&nbsp;</strong> recently?': {'Has fallen (Please document details of fall including how many, when, what happened, any injuries, etc.)',
  'No falls'},
 'Have you met the care team from {{location_name}} who will be taking care of you?': {'Has met team',
  'Has not met team'},
 'How have you been feeling lately?': {'Feeling about the same',
  'Feeling better',
  'Feeling good',
  'Feeling worse'},
 'Do you have any questions about your <strong>&nbsp;

In [18]:
#create columns for each question and answer pair
def ohe_qa_instance(df):
    mlb = MultiLabelBinarizer(sparse_output=True)
    df_copy = df.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df['question_answer_zip']),
                index=df.index,
                columns=mlb.classes_))
    return df_copy
encodedAll = ohe_qa_instance(data)
encodedAll

,stay_id,timestamp,readmission_risk,enacted_emergency_protocol,optional_relation,outcome,call_category_id,is_urgent,is_provider_aware,question_text,...,"([CoreQ] In recommending this facility to your friends and family, how would you rate it overall?, Facility is poor)","([CoreQ] In recommending this facility to your friends and family, how would you rate it overall?, Facility is very good)","([CoreQ] Overall, how would you rate the staff?\r\n<br/>\r\n<br/>\r\n<br/>, Staff is average)","([CoreQ] Overall, how would you rate the staff?\r\n<br/>\r\n<br/>\r\n<br/>, Staff is excellent)","([CoreQ] Overall, how would you rate the staff?\r\n<br/>\r\n<br/>\r\n<br/>, Staff is good)","([CoreQ] Overall, how would you rate the staff?\r\n<br/>\r\n<br/>\r\n<br/>, Staff is poor)","([CoreQ] Overall, how would you rate the staff?\r\n<br/>\r\n<br/>\r\n<br/>, Staff is very good)","({{location_name}} strives for a Perfect 10 in everything we do. Is there anything in your care up to this point that has been less than a Perfect 10? (If less than perfect 10 describe concerns in notes.), Can't think of anything less than perfect 10)","({{location_name}} strives for a Perfect 10 in everything we do. Is there anything in your care up to this point that has been less than a Perfect 10? (If less than perfect 10 describe concerns in notes.), Less than a Perfect 10)","({{location_name}} strives for a Perfect 10 in everything we do. Is there anything in your care up to this point that has been less than a Perfect 10? (If less than perfect 10 describe concerns in notes.), Perfect 10)"
0,1839118,2021-01-14 17:54:13.761181+00,1,f,NaN,1,1.0,f,NaN,"[Today is your pain better, the same, or worse...",...,0,0,0,0,0,0,0,0,0,0
1,1221101,2020-04-30 15:20:58.980727+00,1,f,NaN,1,3.0,f,NaN,"[If you ever need medical care, you can call u...",...,0,0,0,0,0,0,0,0,0,0
2,1220991,2020-04-22 00:13:12.255228+00,1,f,DAUGHTER,1,1.0,f,NaN,[Have you had any <strong>&nbsp;falls&nbsp;</s...,...,0,0,0,0,0,0,0,0,0,0
3,2492049,2021-03-24 16:57:25.837252+00,2,f,DAUGHTER,1,1.0,f,NaN,"[Today is your pain better, the same, or worse...",...,0,0,0,0,0,0,0,0,0,0
4,1602247,2020-10-19 14:29:59.756811+00,1,f,NaN,1,1.0,f,NaN,"[Today is your pain better, the same, or worse...",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489143,550434,2019-01-21 17:13:56.948772+00,1,f,NaN,1,1.0,f,NaN,[Have you had any of the following symptoms - ...,...,0,0,0,0,0,0,0,0,0,0
489144,856557,2019-08-19 15:29:12.512955+00,1,f,NaN,1,1.0,f,NaN,[Have you had any of the following symptoms - ...,...,0,0,0,0,0,0,0,0,0,0
489145,999954,2019-10-17 15:59:47.362389+00,4,f,MOTHER,1,1.0,f,NaN,[Do you have any <strong>&nbsp;concerns&nbsp;<...,...,0,0,0,0,0,0,0,0,0,0
489146,89730,2017-07-11 22:16:41.157076+00,1,f,NaN,1,1.0,f,NaN,"[How have you been feeling lately?, Do you hav...",...,0,0,0,0,0,0,0,0,0,0


In [19]:
def questions_not_asked (data):
    reservoir=list(create_bank(data))
    dummy_data=data
    for i in range(len(reservoir)):
        what_question=reservoir[i]
        question_cols = [col for col in dummy_data.columns if what_question in col]
        dummy_data['not_answered: {}'.format(what_question)]=0
        #not_answered_col.apply(lambda x: 1 if dummy_data[question_cols].max(axis=1) == 0 else 0)
        dummy_data.loc[dummy_data[question_cols].max(axis=1) == 0, ['not_answered: {}'.format(what_question)]]=1
    return dummy_data

In [32]:
#drop questions based on condtion 1
def drop_columns_condition1 (encoded):
    useless_questions=useless_questions_condition_1(encoded)
    dummy_data=encoded
    for i in range(len(useless_questions)):
        what_question=useless_questions[i]
        not_useful_question_cols = [col for col in dummy_data.columns if what_question in col]
        dummy_data=dummy_data.drop(columns=not_useful_question_cols)
    return dummy_data

In [31]:
sample

,stay_id,timestamp,readmission_risk,enacted_emergency_protocol,optional_relation,outcome,call_category_id,is_urgent,is_provider_aware,question_text,...,is_callable,actual_cost,anticipated_cost,lace_score,stay_type,payer,discharge_disposition,referring_location,readmitted,question_answer_zip
300159,1868883,2021-02-02 19:43:38.632557+00,1,f,NaN,1,1.0,f,NaN,"[Today is your pain better, the same, or worse...",...,t,NaN,NaN,NaN,home_health_short_term,NGS,NaN,NaN,0,"[(Today is your pain better, the same, or wors..."
221778,1030576,2020-01-18 03:05:26.354709+00,1,f,NaN,1,1.0,f,NaN,"[Today is your pain better, the same, or worse...",...,t,NaN,NaN,NaN,home_health_short_term,NGS,NaN,NaN,0,"[(Today is your pain better, the same, or wors..."
68443,2924682,2021-07-09 17:10:51.562878+00,2,f,SELF,1,6.0,f,NaN,[Are there any care team members who took extr...,...,t,NaN,NaN,NaN,snf_short_term,Aetna(MAOLP-pdpm),NaN,NaN,0,[(Are there any care team members who took ext...
398235,604442,2019-02-20 17:45:37.714491+00,1,f,NaN,1,1.0,f,NaN,[Have you had any <strong>&nbsp;falls&nbsp;</s...,...,t,NaN,NaN,NaN,home_health_short_term,CIGNA GOVERNMENT SERVICES - (UT & CO HH),NaN,NaN,0,[(Have you had any <strong>&nbsp;falls&nbsp;</...
56768,2334704,2021-03-25 14:49:48.618522+00,1,f,NaN,1,3.0,f,NaN,"[If you ever need medical care, you can call u...",...,t,NaN,NaN,NaN,snf_short_term,BCBSICP,NaN,NaN,0,"[(If you ever need medical care, you can call ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431070,1532979,2020-08-24 19:45:07.563239+00,2,f,NaN,1,3.0,f,t,"[If you ever need medical care, you can call u...",...,t,NaN,NaN,NaN,home_health_short_term,Medicare- Washington,NaN,NaN,0,"[(If you ever need medical care, you can call ..."
476149,1690398,2020-12-31 19:44:01.330959+00,1,f,OTHER RELATIVE,1,3.0,f,NaN,"[If you ever need medical care, you can call u...",...,t,NaN,NaN,NaN,home_health_short_term,MEDICARE PDGM CGS,NaN,NaN,0,"[(If you ever need medical care, you can call ..."
29181,1606202,2020-10-26 22:19:40.490672+00,1,f,NaN,1,2.0,f,NaN,[If you were discharged with a Home Health ser...,...,t,NaN,NaN,NaN,snf_short_term,NaN,NaN,NaN,0,[(If you were discharged with a Home Health se...
301323,851220,2020-01-08 22:55:26.79618+00,1,f,SON,1,3.0,f,NaN,"[If you ever need medical care, you can call u...",...,t,NaN,NaN,NaN,home_health_short_term,UHC OPTUM NETWORK (SYMBII),NaN,NaN,0,"[(If you ever need medical care, you can call ..."


In [25]:
sample=data.sample(500)
dummy_data=ohe_qa_instance(sample)
dummy_data=questions_not_asked(dummy_data)
dummy_data=drop_columns_condition1(dummy_data)
dummy_data
# here's a sample data with our pipeline

,stay_id,timestamp,readmission_risk,enacted_emergency_protocol,optional_relation,outcome,call_category_id,is_urgent,is_provider_aware,question_text,...,not_answered: Have you had any <strong>&nbsp;falls&nbsp;</strong> recently?,not_answered: Are you willing to write a 5 star Google or Yelp review? <em>If yes</em> Can we use your name? (Only ask this question if the patient was overall happy with their care.),not_answered: How likely would you be to recommend {{location_name}} to a friend? (What could {{location_name}} do better?),"not_answered: If you ever need medical care, you can call us. We have a nurse on-call 24 hours a day, 7 days a week. Do you have our <strong>&nbsp;phone number&nbsp;</strong>? {{location_phonenumber}}",not_answered: Did your home health nurse explain your medications and possible side effects?,not_answered: When the home health nurse came to your home did they ask to see all of the prescription and over-the counter medicines you are taking?,"not_answered: Today is your pain better, the same, or worse than it has been?",not_answered: Has your home health team reviewed your medications with you?,not_answered: Did our home care staff call you the night before your admission to arrange a home visit and set up a time convenient for you?,not_answered: Is there anyone in your care team who is taking extra good care of you? (write EMPLOYEE NAME in comments)
300159,1868883,2021-02-02 19:43:38.632557+00,1,f,NaN,1,1.0,f,NaN,"[Today is your pain better, the same, or worse...",...,0,1,1,1,1,1,0,1,1,1
221778,1030576,2020-01-18 03:05:26.354709+00,1,f,NaN,1,1.0,f,NaN,"[Today is your pain better, the same, or worse...",...,0,1,1,1,1,1,0,1,1,1
68443,2924682,2021-07-09 17:10:51.562878+00,2,f,SELF,1,6.0,f,NaN,[Are there any care team members who took extr...,...,1,1,1,1,1,1,1,0,1,1
398235,604442,2019-02-20 17:45:37.714491+00,1,f,NaN,1,1.0,f,NaN,[Have you had any <strong>&nbsp;falls&nbsp;</s...,...,0,1,1,0,1,1,1,1,1,0
56768,2334704,2021-03-25 14:49:48.618522+00,1,f,NaN,1,3.0,f,NaN,"[If you ever need medical care, you can call u...",...,0,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431070,1532979,2020-08-24 19:45:07.563239+00,2,f,NaN,1,3.0,f,t,"[If you ever need medical care, you can call u...",...,0,1,1,1,1,1,1,1,1,1
476149,1690398,2020-12-31 19:44:01.330959+00,1,f,OTHER RELATIVE,1,3.0,f,NaN,"[If you ever need medical care, you can call u...",...,0,1,0,1,1,1,1,1,1,1
29181,1606202,2020-10-26 22:19:40.490672+00,1,f,NaN,1,2.0,f,NaN,[If you were discharged with a Home Health ser...,...,0,1,1,1,1,1,1,1,1,1
301323,851220,2020-01-08 22:55:26.79618+00,1,f,SON,1,3.0,f,NaN,"[If you ever need medical care, you can call u...",...,0,1,1,1,1,1,1,1,1,1


In [29]:
#ultimately this is what our pipeline looks like so far
def pipeline_1(data):
    clean1=ohe_qa_instance(data)
    clean2=questions_not_asked(clean1)
    final_clean=drop_columns_condition1(clean2)
    return final_clean

In [33]:
#and this is the whole
data_with_features=pipeline_1(data)
data_with_features
#after dropping columns with one unique answer, the number of columns went from 1245 to 1145

,stay_id,timestamp,readmission_risk,enacted_emergency_protocol,optional_relation,outcome,call_category_id,is_urgent,is_provider_aware,question_text,...,not_answered: Our hope is that your experience with <b>{{location_name}}</b> is such that you would <b>DEFINITELY</b> recommend us and rate the services you receive from <b>{{location_name}}</b> a 10 on a scale of 1-10. Is there anything up to this point that would prevent you from <b>DEFINITELY</b> recommending <b>{{location_name}}</b> or scoring the services you have received less than 10/10?,not_answered: How has your over all care been up to this point?,not_answered: (IF patient had a positive therapy experience) Would you mind if someone from our therapy department reaches out to assess if you’re an appropriate outpatient candidate?,not_answered: I see that you currently have COPD. How has that been going lately?,not_answered: Have you had any medical procedures since your home health care ended?,not_answered: Are you willing to write a 5 star Google or Yelp review? <em>If yes</em> Can we use your name? (Only ask this question if the patient was overall happy with their care.),"not_answered: If you were discharged with a Home Health service, has the company made contact with you yet?",not_answered: Did your home health nurse explain your medications and possible side effects?,not_answered: What happened that made you go to the hospital?,not_answered: Is there anyone in your care team who is taking extra good care of you? (write EMPLOYEE NAME in comments)
0,1839118,2021-01-14 17:54:13.761181+00,1,f,NaN,1,1.0,f,NaN,"[Today is your pain better, the same, or worse...",...,1,1,1,1,1,1,1,1,1,1
1,1221101,2020-04-30 15:20:58.980727+00,1,f,NaN,1,3.0,f,NaN,"[If you ever need medical care, you can call u...",...,1,1,1,1,1,1,1,1,1,1
2,1220991,2020-04-22 00:13:12.255228+00,1,f,DAUGHTER,1,1.0,f,NaN,[Have you had any <strong>&nbsp;falls&nbsp;</s...,...,1,1,1,1,1,1,1,1,1,1
3,2492049,2021-03-24 16:57:25.837252+00,2,f,DAUGHTER,1,1.0,f,NaN,"[Today is your pain better, the same, or worse...",...,1,1,1,1,1,1,1,1,1,1
4,1602247,2020-10-19 14:29:59.756811+00,1,f,NaN,1,1.0,f,NaN,"[Today is your pain better, the same, or worse...",...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489143,550434,2019-01-21 17:13:56.948772+00,1,f,NaN,1,1.0,f,NaN,[Have you had any of the following symptoms - ...,...,1,1,1,1,1,1,1,1,1,1
489144,856557,2019-08-19 15:29:12.512955+00,1,f,NaN,1,1.0,f,NaN,[Have you had any of the following symptoms - ...,...,1,1,1,1,1,1,1,1,1,1
489145,999954,2019-10-17 15:59:47.362389+00,4,f,MOTHER,1,1.0,f,NaN,[Do you have any <strong>&nbsp;concerns&nbsp;<...,...,1,1,1,1,1,1,1,1,1,0
489146,89730,2017-07-11 22:16:41.157076+00,1,f,NaN,1,1.0,f,NaN,"[How have you been feeling lately?, Do you hav...",...,1,1,1,1,1,1,1,1,1,0


our intentions was to eliminate questions based on their categories' sentiment (is the question is pertaining to feedback of staff, the general health condition of the patient?) however we found out that questions in certain categories we've surveyed before are different from the new data, hence we didn't move forward to drop features based on their call_categories.

below are obsolete funcitons made for reading the sentiment of questions in certain CCIDs

In [ ]:
def data_with_n_cat(data, category):
    filtered_data=data[data['call_category_id']==category]
    return filtered_data

In [ ]:
def check_questions_for_call_category(number, data):
    x = data[data['call_category_id'] == number]['question_text'].value_counts(dropna=False)
    return pd.DataFrame(x).reset_index()
def check_disjoint(number1, number2, data):
    x = check_questions_for_call_category(number1,data)
    y = check_questions_for_call_category(number2,data)
    merged = x.merge(y,how='inner')
    return len(merged) == 0
def make_disjoint_matrix(total_num_of_call_categories, data): 
    matrix = [[0 for x in range(total_num_of_call_categories)] for y in range(total_num_of_call_categories)]
    for i in np.arange(0,total_num_of_call_categories):
        for j in np.arange(0,total_num_of_call_categories):
            matrix[i][j] = check_disjoint(i+1,j+1, data)
    return matrix
def vibe_check_questions(n,data):
    df_n = check_questions_for_call_category(n,data)
    pd.reset_option('display.max_rows',None)
    return df_n
